In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install pymorphy2[fast]
!pip install navec

     |████████████████████████████████| 2.9 MB 5.6 MB/s 
     |████████████████████████████████| 596 kB 56.6 MB/s 
     |████████████████████████████████| 3.3 MB 45.4 MB/s 
     |████████████████████████████████| 56 kB 4.3 MB/s 
     |████████████████████████████████| 895 kB 46.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 8.2 MB 5.6 MB/s 
     |████████████████████████████████| 371 kB 71.5 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=857536 sha256=98d1b8c3068885bebda1eed759a65101eaeb5d553a623ee56abdaa481845258a
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [3]:
import numpy as np
import pymorphy2
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from navec import Navec
from typing import List
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models import KeyedVectors
import scipy.spatial.distance as cos_dist
from tqdm import tqdm

random.seed(44)
morph = pymorphy2.MorphAnalyzer()
nltk.download('punkt')
save_path = '/content/drive/MyDrive'
navec = Navec.load(save_path + '/navec_hudlit_v1_12B_500K_300d_100q.tar')
alphabet = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
categories = ("животные", "музыка", "спорт", "литература")
categories_eng = ("animals", "music", "sport", "literature")

# Открываем файлы с ключевыми словами и убираем units и keywords_tasks из них

with open(save_path + '/units.txt', 'r') as f:
  words = f.read()
  units = set(words.split('\n'))
with open(save_path + '/keywords_tasks.txt', 'r') as f:
  words = f.read()
  keywords_tasks = set(words.split('\n'))

all_nouns_actors = set()
nouns_actors = []
for ind, cat in enumerate(categories_eng):
  with open(save_path + f'/keywords/actors/true_keywords_nouns_actors_{cat}.txt', 'r') as f:
    words = f.read()
  nouns_actors.append([])
  for word in words.split('\n')[:-1]:
    if not word in units:
      nouns_actors[ind].append(word)
  all_nouns_actors |= set(nouns_actors[ind])

all_nouns = set()
nouns = []
for ind, cat in enumerate(categories_eng):
  with open(save_path + f'/wow_keywords/wow_keywords_{cat}.txt', 'r') as f:
    words = f.read()
  nouns.append([])
  for word in words.split('\n')[:-1]:
    if (not word in units) and (not word in keywords_tasks):
      nouns[ind].append(word)
  all_nouns |= set(nouns[ind])

In [5]:
# Делаем маски
class MaskCreator():

  def __init__(self):
    self.tokenizer = nltk.WordPunctTokenizer()
    self.bigram_mod = gensim.models.Phrases.load(save_path + '/bigram_model.pkl')

  def make_bigrams(self, doc):
      return self.bigram_mod[doc]

  def mask(self, text, category=4, make_bigrams=False):
    masks_dict = []
    tokens = self.tokenizer.tokenize(text.lower())
    
    if make_bigrams:
      tokens_normal = [morph.parse(w)[0].normal_form for w in tokens]
      tokens_bigrammed = self.make_bigrams(tokens_normal)
      
      if len(tokens_bigrammed) < len(tokens):
        ind_go = 0
        for i in range(len(tokens_bigrammed)):
          if tokens_normal[ind_go] != tokens_bigrammed[i]:
            tokens = tokens[:ind_go] + [tokens_bigrammed[i]] + tokens[ind_go+2:]
            ind_go += 2
          else:
            ind_go += 1

    if category == 4:
      now_keywords = all_nouns
    else:
      now_keywords = nouns[category]

    prev_words = []
    for ind, token in enumerate(tokens):
      word = morph.parse(token.lower())[0].normal_form
      if word in now_keywords:
        if word not in masks_dict:
          masks_dict.append(word)
        prev_words.append(tokens[ind])
        tokens[ind] = 'mask' + str(masks_dict.index(word, 0))
    text = nltk.tokenize.treebank.TreebankWordDetokenizer().detokenize(tokens)
    return text, masks_dict, prev_words

In [90]:
'''check_words = [['животные', 'животное', 'собака', 'кошка', 'млекопитающее', 'птица', 'зверь', 'хищник', 'паразит', 'бактерия'],
               ['музыка', 'нота', 'ноты', 'песня', 'октава', 'музыкант', 'звучать', 'петь', 'пианино', 'гитара'],
               ['спорт', 'соревнование', 'побеждать', 'победа', 'матч', 'стадион', 'спортсмен', 'болельщик', 'сборная', 'выиграть'],
               ['литература', 'книга', 'страница', 'писатель', 'чтение', 'читать', 'читатель', 'газета', 'библиотека', 'книжка']]'''
check_words = [['животные', 'животное', 'зверь'],
               ['музыка', 'нота', 'звучать'],
               ['спорт', 'побеждать', 'спортсмен'],
               ['литература', 'книга', 'читать']]

In [91]:
# Создаём качественные ключевые слова

all_keywords = [[] for i in range(4)]
already_in_keywords = set()
for word in tqdm(navec.vocab.word_ids.keys()):
  word_normal = morph.parse(word)[0].normal_form
  if (not word[0] in alphabet) or word_normal in already_in_keywords:
    continue
  min_check_words = [1, 1, 1, 1]
  max_check_words = [0, 0, 0, 0]
  for ind_cat in range(4):
    for check_word in check_words[ind_cat]:
      now_cos_dist = cos_dist.cosine(navec[word], navec[check_word])
      min_check_words[ind_cat] = min(min_check_words[ind_cat], now_cos_dist)
      max_check_words[ind_cat] = max(max_check_words[ind_cat], now_cos_dist)
  ind_min = -1
  for ind_cat in range(4):
    max_lower_mins = True
    for last_ind_cat in range(4):
      if ind_cat == last_ind_cat:
        continue
      if max_check_words[ind_cat] > min_check_words[last_ind_cat]: # Если маскимальное расстояние от слова до всех векторов из одной категории меньше, чем минимальное расстояние до всех остальных, то считаем его ключевым
        max_lower_mins = False
        break
    if max_lower_mins:
      ind_min = ind_cat
      break
  if ind_min != -1:
    all_keywords[ind_min].append(word_normal)
    already_in_keywords.add(word_normal)

100%|██████████| 500002/500002 [11:32<00:00, 721.63it/s]


In [92]:
for ind, cat in enumerate(categories_eng):
  with open(save_path + f'/wow_keywords/wow_keywords_{cat}.txt', 'w') as f:
    f.write('\n'.join(list(set(all_keywords[ind])|set(nouns[ind]))))
    #f.write('\n'.join(all_keywords[ind]))

In [32]:
def nearest_word(ideal_dist, list_with_words, ban_words=[]): # Ближайшее слово из (list_with_words без ban_words) к вектору ideal_dist
  min_val = 1
  min_new_word = ''
  for new_word in list_with_words:
    if (not new_word in navec) or (new_word in ban_words):
      continue
    now_dist = cos_dist.cosine(navec[new_word], ideal_dist)
    if now_dist < min_val:
      min_val = now_dist
      min_new_word = new_word
  return min_val, min_new_word

In [33]:
def translate(sentence, category_from, category_to, all_print=False):
  if all_print:
    print(sentence)
  sentence_with_masks, masks, prev_words = MaskCreator().mask(sentence, category_from)
  if all_print:
    print(sentence_with_masks, masks)
  
  new_masks = ['' for i in range(len(masks))]
  words_in_sentence = word_tokenize(sentence_with_masks)
  get_first_mask = False
  now_ind_to_prev_ind = {}
  count_masks = 0
  for ind, word in enumerate(words_in_sentence):
    if word[:4] == 'mask':
      now_ind_to_prev_ind[ind] = count_masks
      count_masks += 1

  # Подбираем первое слово (подлежащее)
  for word in words_in_sentence:
    if word[:4] == 'mask' and (masks[int(word[4:])] in nouns_actors[category_from]) and (masks[int(word[4:])] in navec):
      first_mask = navec[masks[int(word[4:])]]
      min_val, min_new_word = nearest_word(first_mask - navec[categories[category_from]] + navec[categories[category_to]], nouns_actors[category_to])
      new_word = min_new_word
      new_first_mask = navec[new_word]
      new_masks[int(word[4:])] = new_word
      get_first_mask = True
      break

  # Подбираем все остальные слова
  for ind, word in enumerate(words_in_sentence):
    if word[:4] == 'mask':
      if not masks[int(word[4:])] in navec:
        new_masks[int(word[4:])] = masks[int(word[4:])]
      # Если первое слово ещё не выбрано
      if (not get_first_mask) and (masks[int(word[4:])] in navec):
        first_mask = navec[masks[int(word[4:])]]
        min_val, min_new_word = nearest_word(first_mask - navec[categories[category_from]] + navec[categories[category_to]], nouns[category_to])
        new_word = min_new_word
        new_first_mask = navec[new_word]
        new_masks[int(word[4:])] = new_word
        get_first_mask = True
      elif new_masks[int(word[4:])] == '' and (masks[int(word[4:])] in navec):
        min_val, min_new_word = nearest_word(new_first_mask - (first_mask - navec[masks[int(word[4:])]]), nouns[category_to], new_masks)
        if min_val > 0.64: # Если полученное слово не очень похоже на слово, которе должно там стоять, не меняем это слово
          min_new_word = masks[int(word[4:])]
        new_masks[int(word[4:])] = min_new_word
      put_word = new_masks[int(word[4:])]
      prev_word = prev_words[now_ind_to_prev_ind[ind]]
      case_prev_word = morph.parse(prev_word)[0].tag.case
      number_prev_word = morph.parse(prev_word)[0].tag.number
      # Ставим в исходную форму, если можем (в форму в первом предложении)
      try:
        words_in_sentence[ind] = morph.parse(put_word)[0].inflect({number_prev_word, case_prev_word})[0]
      except:
        words_in_sentence[ind] = put_word

  sentence = TreebankWordDetokenizer().detokenize(words_in_sentence)

  return sentence

In [14]:
#sentence = '4.30. У слона пульс 20 ударов в минуту, а у паука на 40 ударов в минуту больше. Какой пульс у паука?'
#sentence = '   1.1. Летом в спортивный лагерь ходили 50 детей, из них 9 девочек. Сколько мальчиков ходили в спортивный лагерь?'
#sentence = '   1.4. Во время летних соревнований по плаванию ребята посетили бассейн. Длина плавательной дорожки в бассейне 25 м. После того как первый участник соревнований проплыл часть дорожки, ему осталось проплыть 10 м. Сколько метров уже проплыл участник соревнований?'
#sentence = 'Используя данные круговой диаграммы, реши задачу. На диаграмме представлены данные о турпоходе группы. Сколько километров прошла группа в четвёртый день? Введи в поле ответа число без единиц измерения. При необходимости ввести десятичную дробь, разделяй её целую и дробную части запятой, без пробелов.'
#sentence = 'Во дворе кот Геннадий охотился на напыщенного толстого голубя. (1) крадущегося кота была равна 1 м/с. Но голубь оказался не так прост и скоро, через (2), заметил приближающегося Геннадия. Птица взлетела на крышу гаража высотой в (3), чтобы отвязаться от кота. Но тот быстро преодолел (4) до этого строения в 5 метров, прыгнул наверх и почти ухватил пернатого за хвост! Голубь, конечно, не стерпел такой наглости и взлетел со скоростью (5). Обиженный Геннадий посмотрел некоторое (6) на улетающую птицу и спрыгнул вниз. Какова длина траектории (в метрах) кота Геннадия за описанную утреннюю прогулку?'
#sentence = 'У Миши было 3 три мячика. Два из них он отдал Даше. Сколько мячиков осталось у Миши?'
sentence = 'Как ты уже знаешь, для того чтобы транспортное средство перемещалось с большой скоростью, ему необходимо ускоряться. Будь то самолет, поезд или автомобиль, люди часто готовы рисковать своей жизнью ради победы в гонке или мирового рекорда. Подготовь выступление о 10 самых интересных быстрых транспортных средствах, когда-либо созданных человеком. Оцени ускорение, с которым они разгонялись до максимальной скорости. Результат представь в виде презентации.'
category_from = 2
category_to = 1

print(translate(sentence, category_from, category_to, True))

Как ты уже знаешь, для того чтобы транспортное средство перемещалось с большой скоростью, ему необходимо ускоряться. Будь то самолет, поезд или автомобиль, люди часто готовы рисковать своей жизнью ради победы в гонке или мирового рекорда. Подготовь выступление о 10 самых интересных быстрых транспортных средствах, когда-либо созданных человеком. Оцени ускорение, с которым они разгонялись до максимальной скорости. Результат представь в виде презентации.
как ты уже знаешь, для того чтобы транспортное средство перемещалось с большой скоростью, ему необходимо ускоряться . будь то самолет, поезд или mask0, люди часто готовы рисковать своей жизнью ради mask1 в mask2 или мирового mask3 . подготовь mask4 о 10 самых интересных mask5 транспортных средствах, когда - либо созданных человеком . оцени ускорение, с которым они разгонялись до mask6 скорости . результат представь в виде презентации. ['автомобиль', 'победа', 'гонка', 'рекорд', 'выступление', 'быстрый', 'максимальный']
как ты уже знаешь, 

In [16]:
df_tasks = pd.read_csv(save_path + '/dataset_disclosed.csv', sep=';')
df_ans = pd.read_csv(save_path + '/test_finals.csv', sep=';')

In [ ]:
df_tasks

In [ ]:
df_ans

In [ ]:
for ind in tqdm(range(len(df_tasks['task']))):
  category_from = categories.index(df_tasks['category'][ind])
  for category_to in range(4):
    if category_to == category_from:
      df_ans[categories_eng[category_to]][ind] = df_tasks['task'][ind]
    else:
      df_ans[categories_eng[category_to]][ind] = translate(df_tasks['task'][ind], category_from, category_to)

  0%|          | 0/514 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
 74%|███████▍  | 380/514 [42:47<19:37,  8.78s/it]

In [ ]:
df_ans.to_csv(save_path + '/ans.csv', index=False)